In [10]:
import torch
import keras
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import data_loader
import autoencoder as ae

In [11]:
torch.cuda.is_available()

True

In [12]:
trainx, devx, testx, trainy, devy, testy = data_loader.load_all_classic_random_split(flatten=False)

Processing albert
Processing canon_12_5
Processing Chen_12_7
Processing daniel
Processing haobin_11_22
Processing isa_12_5
Processing janet
Processing joanne
Processing jq_12_6
Processing kelly_11_7
Processing kevin_11_7
Processing ruocheng
Processing russell_11_20_stand
Processing russell_11_7
Processing russell_random_12_7
Processing solomon
Processing wenzhou_12_5
Processing yiheng_11_30
Processing yiheng_12_5
Processing yongxu_11_30
Processing Zhaoye_12_7
Splitting out test set
Splitting out dev and train set


In [13]:
trainx, trainy = data_loader.augment_train_set(trainx, trainy, augment_prop=3, is_flattened=False)
print(trainx.shape, devx.shape, testx.shape, trainy.shape, devy.shape, testy.shape)

Augmenting TRAIN set with proportion 3
(33280, 100, 3) (1040, 100, 3) (1041, 100, 3) (33280,) (1040,) (1041,)


In [16]:
trainx = ae.ae_denoise(trainx[:,:,0], trainx[:,:,1], trainx[:,:,2])
devx = ae.ae_denoise(devx[:,:,0], devx[:,:,1], devx[:,:,2])
testx = ae.ae_denoise(testx[:,:,0], testx[:,:,1], testx[:,:,2])
print(trainx.shape, devx.shape, testx.shape, trainy.shape, devy.shape, testy.shape)

Train on 33280 samples
Epoch 1/10
33280/33280 [==============================] - 2s 61us/sample - loss: 0.6466
Epoch 2/10
33280/33280 [==============================] - 2s 46us/sample - loss: 0.5772
Epoch 3/10
33280/33280 [==============================] - 2s 47us/sample - loss: -1.3196
Epoch 4/10
33280/33280 [==============================] - 2s 47us/sample - loss: -20.2893
Epoch 5/10
33280/33280 [==============================] - 2s 47us/sample - loss: -85.4220
Epoch 6/10
33280/33280 [==============================] - 2s 46us/sample - loss: -226.4266
Epoch 7/10
33280/33280 [==============================] - 2s 47us/sample - loss: -481.2242
Epoch 8/10
33280/33280 [==============================] - 2s 46us/sample - loss: -874.3196
Epoch 9/10
33280/33280 [==============================] - 2s 47us/sample - loss: -1434.4918
Epoch 10/10
33280/33280 [==============================] - 2s 46us/sample - loss: -2268.6696
Train on 33280 samples
Epoch 1/10
33280/33280 [===========================

KeyboardInterrupt: 

In [ ]:
BATCH_SIZE = 500

def get_dataloader(x, y, batch_size):
    dataset = [(x[i].T, y[i]) for i in range(y.shape[0])]
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

trainloader = get_dataloader(trainx, trainy, BATCH_SIZE)
devloader = get_dataloader(devx, devy, BATCH_SIZE)
testloader = get_dataloader(testx, testy, BATCH_SIZE)

In [ ]:
sample_size, num_feature, num_channel = trainx.shape
print(sample_size, num_feature, num_channel)

In [ ]:
def acc(data_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            x, y = data
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()

            outputs = net(x.float())
            _, predicted = torch.max(outputs.data, 1)

            w = torch.sum((predicted - y) != 0).item()
            r = len(y) - w
            correct += r
            total += len(y)
    return correct / total

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers):
        super(Net, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 26, bias = True)

    def forward(self, x):
        init_h = torch.randn(self.n_layers, x.shape[0], self.hidden_dim).cuda()
        init_c = torch.randn(self.n_layers, x.shape[0], self.hidden_dim).cuda()
        x = x.permute(0, 2, 1)
        out, _ = self.lstm(x, (init_h, init_c))
        # print("inter: ", out.shape)
        out = self.fc(out[:,-1,:])
        # print("out: ", out.shape)
        return out

net = Net(num_channel, 100, 5)
if torch.cuda.is_available():
    net.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.0001)

for epoch in range(500):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        print(f'{i if i%20==0 else ""}.', end='')

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize

        outputs = net(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

    trainacc = acc(trainloader)
    devacc = acc(devloader)

    print('')
    print(f'Epoch {epoch} trainacc={trainacc} devacc={devacc}')

print('Finished Training')

In [ ]:
print(acc(testloader))